<a href="https://colab.research.google.com/github/rzunick/Dominant-Color-Detection/blob/main/COCO_2017_Color_Dataset_Generator_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Dependencies

In [ ]:
# Importing FiftyOne Library for Dataset Management
!pip install fiftyone
!pip install -U scikit-learn fiftyone

# FiftyOne dependencies
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import os
from PIL import Image

# MultiModal LLM Dependencies
!pip install open_clip_torch
!pip install timm --upgrade

# Color Analysis libraries
from skimage import io, color
from skimage.transform import resize
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Exporting Dataset
!pip install -q zipfile36
import shutil
from google.colab import files


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

# Importing the Coco 2017 Dataset

In [ ]:
# Load a subset of the COCO dataset
dataset = foz.load_zoo_dataset(
    "coco-2017",
    #split="validation",
    label_types=["detections"],
    #max_samples=100,
)

dataset.name = "Coco-original"

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


INFO:fiftyone.utils.coco:Downloading annotations to '/root/fiftyone/coco-2017/tmp-download/annotations_trainval2017.zip'


 100% |██████|    1.9Gb/1.9Gb [3.2s elapsed, 0s remaining, 1.2Gb/s]        


INFO:eta.core.utils: 100% |██████|    1.9Gb/1.9Gb [3.2s elapsed, 0s remaining, 1.2Gb/s]        


Extracting annotations to '/root/fiftyone/coco-2017/raw/instances_train2017.json'


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'test'


 100% |█████████████| 40670/40670 [13.9s elapsed, 0s remaining, 2.9K samples/s]      


INFO:eta.core.utils: 100% |█████████████| 40670/40670 [13.9s elapsed, 0s remaining, 2.9K samples/s]      


Dataset 'coco-2017' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017' created


In [ ]:
#session = fo.launch_app(dataset)

In [ ]:
# Check the first few samples to see attributes formatting
# for sample in dataset.take(2):
    # print(sample.id, hasattr(sample, 'ground_truth'), sample.ground_truth if hasattr(sample, 'ground_truth') else "No ground_truth")

# Generating New Dataset - Segmenting and resizing images

In [ ]:
# Directory to save the cropped and resized images
os.makedirs("Coco-v2", exist_ok=True)

# Initialize variables for the new dataset in COCO format
images = []
annotations = []
annotation_id = 1

# Mapping of class names to category IDs
category_id_map = {cls: idx + 1 for idx, cls in enumerate(dataset.default_classes)}

for sample in dataset:
    img = Image.open(sample.filepath)
    # Ensure 'ground_truth' exists and is not None, and it contains 'detections'
    if hasattr(sample, 'ground_truth') and sample.ground_truth is not None and sample.ground_truth.detections:
        for detection in sample.ground_truth.detections:
            # Extract bounding box coordinates
            bbox = detection.bounding_box
            x, y, w, h = bbox[0] * img.width, bbox[1] * img.height, bbox[2] * img.width, bbox[3] * img.height
            cropped_img = img.crop((x, y, x + w, y + h))

            # Resize the cropped image to 224x224
            resized_img = cropped_img.resize((224, 224), Image.LANCZOS)  # Use LANCZOS for high-quality downsampling

            # Save the resized image
            resized_img_path = os.path.join("Coco-v2", f"{sample.id}_{detection.label}_{annotation_id}.jpg")
            resized_img.save(resized_img_path)

            # Create new image and annotation entries
            images.append({
                "id": annotation_id,
                "file_name": resized_img_path,
                "width": 224,
                "height": 224
            })

            annotations.append({
                "id": annotation_id,
                "image_id": annotation_id,
                "category_id": category_id_map[detection.label],
                "bbox": [0, 0, 224, 224],
                "area": 224 * 224,
                "iscrowd": 0
            })

            annotation_id += 1

# Create a new dataset from these images and annotations
new_dataset_dict = {
    "images": images,
    "annotations": annotations,
    "categories": [{'id': id, 'name': name} for name, id in category_id_map.items()]
}

# Save this dict to a JSON file to load as a COCO dataset in FiftyOne
import json
with open("Coco-v2.json", "w") as f:
    json.dump(new_dataset_dict, f)

In [ ]:
# Load your new dataset into FiftyOne
new_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path="/content/",
    labels_path="/content/Coco-v2.json"
)
new_dataset.name = "Coco-v2"

 100% |███████████| 896782/896782 [24.9m elapsed, 0s remaining, 286.3 samples/s]      


INFO:eta.core.utils: 100% |███████████| 896782/896782 [24.9m elapsed, 0s remaining, 286.3 samples/s]      


# Generating Dominant Color Labels - Using MultiModal LLM

In [ ]:
model = foz.load_zoo_model(
    "clip-vit-base32-torch",
    text_prompt="The color of the object is",
    classes=["Red","Yellow","Blue","Orange","Green","Purple","Black","White","Gray","Pink"],
)

new_dataset.apply_model(model, label_field="color_predictions")

INFO:fiftyone.core.models:Downloading model from 'https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt'...


 100% |██████|    2.6Gb/2.6Gb [3.0s elapsed, 0s remaining, 925.8Mb/s]       


INFO:eta.core.utils: 100% |██████|    2.6Gb/2.6Gb [3.0s elapsed, 0s remaining, 925.8Mb/s]       


INFO:fiftyone.utils.clip.zoo:Downloading CLIP tokenizer...


 100% |█████|   10.4Mb/10.4Mb [64.7ms elapsed, 0s remaining, 160.0Mb/s]     


INFO:eta.core.utils: 100% |█████|   10.4Mb/10.4Mb [64.7ms elapsed, 0s remaining, 160.0Mb/s]     


 100% |███████████| 896782/896782 [3.1h elapsed, 0s remaining, 77.1 samples/s]      


INFO:eta.core.utils: 100% |███████████| 896782/896782 [3.1h elapsed, 0s remaining, 77.1 samples/s]      


In [ ]:
#session = fo.launch_app(new_dataset)

# Filtering by Confidence Score

In [ ]:
filtered_dataset = new_dataset.filter_labels("color_predictions", F("confidence") > 0.3)

In [ ]:
#session = fo.launch_app(filtered_dataset)

# Class Balancing

In [ ]:
# 1. Identify the class with the lowest quantity of samples
class_counts = filtered_dataset.count_values("color_predictions.label")
min_class_count = min(class_counts.values())
min_class = min(class_counts, key=class_counts.get)

print(f"Lowest count class: {min_class} with {min_class_count} samples")

Lowest count class: Purple with 17267 samples


In [ ]:
# 2 & 3. Balance the dataset
balanced_dataset = fo.Dataset()

for label in class_counts.keys():
    # Get samples for this class
    class_samples = filtered_dataset.match(F("color_predictions.label") == label)

    # Sort by confidence score in descending order
    sorted_samples = class_samples.sort_by("color_predictions.confidence", reverse=True)

    # Take only the top min_class_count samples
    top_samples = sorted_samples[:min_class_count]

    # Add to the balanced dataset
    balanced_dataset.add_samples(top_samples)

# Print the new class distribution
new_class_counts = balanced_dataset.count_values("color_predictions.label")
print("New class distribution:")
for label, count in new_class_counts.items():
    print(f"{label}: {count}")

# Print the total number of samples in the balanced dataset
print(f"Total samples in balanced dataset: {len(balanced_dataset)}")

 100% |█████████████| 17267/17267 [55.7s elapsed, 0s remaining, 388.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████| 17267/17267 [55.7s elapsed, 0s remaining, 388.2 samples/s]      


 100% |█████████████| 17267/17267 [55.3s elapsed, 0s remaining, 382.1 samples/s]      


INFO:eta.core.utils: 100% |█████████████| 17267/17267 [55.3s elapsed, 0s remaining, 382.1 samples/s]      


 100% |█████████████| 17267/17267 [53.4s elapsed, 0s remaining, 382.1 samples/s]      


...

 100% |█████████████| 17267/17267 [52.6s elapsed, 0s remaining, 390.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████| 17267/17267 [52.6s elapsed, 0s remaining, 390.2 samples/s]      


 100% |█████████████| 17267/17267 [54.1s elapsed, 0s remaining, 388.6 samples/s]      


INFO:eta.core.utils: 100% |█████████████| 17267/17267 [54.1s elapsed, 0s remaining, 388.6 samples/s]      


New class distribution:
Red: 17267
Purple: 17267
Green: 17267
Blue: 17267
Gray: 17267
Yellow: 17267
Pink: 17267
White: 17267
Orange: 17267
Black: 17267
Total samples in balanced dataset: 172670


In [ ]:
#session = fo.launch_app(balanced_dataset)

# Exporting COCO 2017 Color Dataset

In [ ]:
# The Dataset or DatasetView containing the samples you wish to export
export_dataset = balanced_dataset

# The directory to which to write the exported dataset
export_dir = "/content/data/export_1"

# The name of the sample field containing the label that you wish to export
# Used when exporting labeled datasets (e.g., classification or detection)
label_field = "color_predictions"

# The type of dataset to export
# Any subclass of `fiftyone.types.Dataset` is supported
dataset_type = fo.types.ImageClassificationDirectoryTree

# Export the dataset
export_dataset.export(
    export_dir=export_dir,
    dataset_type=dataset_type,
    label_field=label_field,
)

In [ ]:
# Define the path to your folder
folder_path = export_dir  # Replace with the actual path to your folder
output_filename = 'export_1.zip'

# Create a ZIP file of the folder
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_path)

# Download the ZIP file
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>